In [ ]:
import argparse
import pexpect

# Parse command line arguments
parser = argparse.ArgumentParser(description='Download files from SFTP server')
parser.add_argument('username', type=str, help='SFTP server username')
parser.add_argument('password', type=str, help='SFTP server password')
parser.add_argument('remote_dir', type=str, help='Remote directory to download from')
parser.add_argument('file_list', type=str, help='File containing list of files to download')
parser.add_argument('local_dir', type=str, help='Local directory to download to')
parser.add_argument('hostname', type=str, help='Remote SFTP server hostname')
parser.add_argument('port', type=int, help='Remote SFTP server port number')

# args = parser.parse_args()
# Pass the arguments to the script for testing
args = parser.parse_args(['USER', 'PASSWORD', '/servdata/support/maprpatches/v7.2.0/rpm/', 'ebf_file_list.txt', '/root/downloads', 'REMOTE.SFTP.SERVER', '22'])

# Read the list of files to download
with open(args.file_list, 'r') as f:
    files_to_download = f.read().splitlines()

# SFTP command to download files
sftp_command = f"sftp -o Port={args.port} {args.username}@{args.hostname}:{args.remote_dir}/{{}} {args.local_dir}/"

# Download each file in the list
for file in files_to_download:
    print('Downloading file: ' + file)
    # Spawn a new process to execute the SFTP command
    sftp_process = pexpect.spawn(sftp_command.format(file))
    # Wait for the password prompt and send the password
    sftp_process.expect("password:")
    sftp_process.sendline(args.password)
    # Wait for the download to complete and print the output
    sftp_process.expect(pexpect.EOF, timeout=360)
    print(sftp_process.before.decode())

# Print a message indicating that the download is complete
print(f"Downloaded {len(files_to_download)} files to {args.local_dir}")